In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests 
import io
import seaborn as sns
import scipy as sp

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize

import string
from nltk import word_tokenize
from nltk.corpus import stopwords

from bs4 import BeautifulSoup

nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 33.8 MB/s 
     |████████████████████████████████| 596 kB 37.0 MB/s 
     |████████████████████████████████| 56 kB 5.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.8 MB/s 
     |████████████████████████████████| 895 kB 31.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer,BertModel, BertConfig
import torch

### User Mapping

In [ ]:
orig_members = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_members.csv')

In [ ]:
sorted_member_id = orig_members['member_id'].sort_values()
member_id_map ={}
start_id = 1
for id in sorted_member_id:
  member_id_map[id] = start_id
  start_id = start_id + 1

### Posts

In [ ]:
orig_posts = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_posts.csv')

In [ ]:
orig_posts.drop(orig_posts.columns[[0,1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]], axis =1, inplace=True)

In [ ]:
orig_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139068 entries, 0 to 139067
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   author_id  139068 non-null  int64 
 1   post       139068 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.1+ MB


### Functions

In [ ]:
def preprocess_text(text):


    soup = BeautifulSoup(text, "lxml")
    text = soup.body.text

    # Get a list of sentences, removes "\n"
    lst = sent_tokenize(text)
    text = " ".join(lst)

    text = text.replace("\n", "")

    # Preprocess using NLTK
    text = text.lower()

    # Remove Punctuation
    text_p = "".join([char for char in text if char not in string.punctuation])

    # Get a List of words 
    words = word_tokenize(text_p)

    words = " ".join(words)

    # Remove stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]

    ret = " ".join(filtered_words)

    return ret

In [ ]:
def getBERT(post):

# Get the embedding
  inputs = tokenizer(post, return_tensors="pt")
  outputs = model(**inputs)
  last_hidden_states = outputs.last_hidden_state

# Take the average
  n_tensor = last_hidden_states.shape[1]
  vector = (last_hidden_states.sum(axis=1)/n_tensor)

  return vector

In [ ]:
def get_word_split(text1):
  l_total = []
  l_partial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150 + 1
  else: 
    n = 1
  # print(n)
  for w in range(n):
    if w == 0:
      l_partial = text1.split()[:200]
      l_total.append(" ".join(l_partial))
    else:
      l_partial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_partial))
  return l_total

### Variables

In [ ]:
embedding = np.zeros((orig_members.shape[0], 768))
# embedding_sparse = sp.sparse.load_npz('/content/drive/MyDrive/text gcn/node_feature/ironmarch/posts_embedding/chunk/v26.npz')
# embedding = embedding_sparse.todense()
count = np.zeros(orig_members.shape[0])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Iterations

In [ ]:
for i, row in orig_posts.iterrows():
  row = orig_posts.iloc[i]
  author_id = row['author_id']
  if author_id not in member_id_map:
          continue
  id = member_id_map[author_id]
  doc = row['post']

  clean_doc = preprocess_text(doc)
  clean_chunks = get_word_split(clean_doc)

  
  embed_doc = np.zeros((1, 768))

  for chunkNo, chunk in enumerate(clean_chunks):
    embed_chunk = getBERT(chunk)
    embed_chunk = embed_chunk.detach().numpy()
    embed_doc = (embed_doc*chunkNo + embed_chunk)/(chunkNo+1)


  embedding[id-1] = (count[id-1]*embedding[id-1] + embed_doc)/(count[id-1]+1)
  count[id-1] = count[id-1]+1

  if i%5000==0:
    num = int(i/5000)
    embedding_sparse = sp.sparse.csr_matrix(embedding)
    sp.sparse.save_npz("/content/drive/MyDrive/text gcn/node_feature/ironmarch/posts_embedding/chunk/v{}".format(num+1), embedding_sparse)
    np.save("/content/drive/MyDrive/text gcn/node_feature/ironmarch/posts_embedding/chunk/count_v{}".format(i+1), count)
    # np.save("/content/drive/MyDrive/text gcn/node_feature/ironmarch/posts_embedding/chunk/v{}".format(num+1), embedding)



/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.youtube.com/watch?v=ukY8XZBRYGo&amp;NR=1" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.youtube.com/watch?v=TWMFSUj2jfU" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.youtube.com/watch?v=sYo8z-PtryM&amp;feature=related" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behi

In [ ]:
embedding_sparse = sp.sparse.csr_matrix(embedding)
sp.sparse.save_npz("/content/drive/MyDrive/text gcn/node_feature/ironmarch/posts_embedding/chunk/final", embedding_sparse)